In [1]:
import fasttext
from sklearn import metrics

In [2]:
model = fasttext.load_model("fasttext-malaya.ftz")

In [3]:
model.predict(['na'], k = 6)[0][0]

['__label__other', '__label__malay', '__label__eng']

In [4]:
import json

with open('train-test.json') as fopen:
    train_test = json.load(fopen)
    
train_test.keys()

dict_keys(['train_X', 'test_X', 'train_Y', 'test_Y'])

In [5]:
test_X, test_Y = train_test['test_X'], train_test['test_Y']

In [6]:
model.predict(test_X[0])[0][0].replace('__label__', ''), test_Y[0]

('eng', 'eng')

In [7]:
results = model.predict(test_X[:2])[0]
[r[0].replace('__label__', '') for r in results], test_Y[:2]

(['eng', 'ind'], ['eng', 'ind'])

In [8]:
results[0]

['__label__eng']

In [9]:
from tqdm import tqdm

batch_size = 128
predicted_Y, actual_Y = [], []

for i in tqdm(range(0, len(test_X), batch_size)):
    index = min(i + batch_size, len(test_X))
    batch_x = test_X[i: index]
    batch_x = [s.replace('\n','').replace('\t', '') for s in batch_x]
    results = model.predict(batch_x)[0]
    predicted_Y.extend([r[0].replace('__label__', '') for r in results])
    actual_Y.extend(test_Y[i: index])

100%|██████████| 36951/36951 [00:38<00:00, 948.22it/s] 


In [10]:
len(predicted_Y) == len(actual_Y)

True

In [11]:
print(
    metrics.classification_report(
        actual_Y,
        predicted_Y,
        digits = 5
    )
)

              precision    recall  f1-score   support

         eng    0.94014   0.96750   0.95362    553739
         ind    0.97290   0.97316   0.97303    576059
       malay    0.98674   0.95262   0.96938   1800649
    manglish    0.96595   0.98417   0.97498    181442
       other    0.98454   0.99698   0.99072   1428083
       rojak    0.81149   0.91650   0.86080    189678

    accuracy                        0.97002   4729650
   macro avg    0.94363   0.96515   0.95375   4729650
weighted avg    0.97111   0.97002   0.97028   4729650

